In [1]:
# Libraries needed: 
import time
import pickle
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate
from dsgtools.reporting import col_summary
from dsgtools import azure

Bad pipe message: %s [b'E\x14\xee\r_4g\xb8\xc9m\xbc\xf2\xa0s\x05\x9d\x85\xcf DDs \xe2\xc8sI\xbe\xb3\x0fq\xa3\xdf\x15\x08']
Bad pipe message: %s [b"\x1f\x045\xaah\xb54\x86\xeb\x0c\xec1\xa8@\x84\xf8\x07\xdb\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1"]
Bad pipe message: %s [b'$\xdeo\xc6\x10lb\xc3uR\x90\xab\xf22\xbe\xbf\x03&\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a']
Bad pipe message: %s [b'\x9d7bq\x1f@\x9f#\xc2\xe4\x87\t,M\x80\x96\x93z\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0

## Sample preparation

#### SBFE Sample

In [4]:
path = "Analytics/Personal Folders/liuwei01/2023/ARMBS_ticket/2390_SOS/combined_prod_liuwei01_37119_customer_input_W20230221-140008.csv"
SBFE_combined = pd.read_adls(path, reader = pd.read_csv,  encoding='iso-8859-1', dtype = str)
print(SBFE_combined.shape)

(981928, 42)


In [5]:
SBFE_combined["blank_ct"] = SBFE_combined[['businessname', 'businessaddress', 'businesscity', 'businessstate']].isnull().sum(axis = 1)
freq(SBFE_combined["blank_ct"])

,Count,Pct,Cuml Count,Cuml Pct
blank_ct,,,,
0,891481.0,0.907888,891481.0,0.907888
1,33132.0,0.033742,924613.0,0.941630
2,41404.0,0.042166,966017.0,0.983796
3,3845.0,0.003916,969862.0,0.987712
4,12066.0,0.012288,981928.0,1.000000


In [6]:
SBFE_combined = SBFE_combined[SBFE_combined["blank_ct"] == 0]
keep = ['transactionid', 'dateadded', 'businessname', 'businessaddress', 'businesscity','businessstate', 'businesszip', 'businessphone',]
SBFE_combined = SBFE_combined[keep]
SBFE_combined = SBFE_combined.sort_values(by = ['businessname', 'businessaddress', 'businesscity','businessstate', 'dateadded'])
SBFE_combined = SBFE_combined.drop_duplicates(subset = ['businessname', 'businessaddress', 'businesscity','businessstate'], 
                                              keep = "last", ignore_index = True)
print(SBFE_combined.shape)

(795958, 8)


In [7]:
freq(SBFE_combined["dateadded"].astype(str).str.slice(0, 6))

,Count,Pct,Cuml Count,Cuml Pct
dateadded,,,,
202210,167259.0,0.210135,167259.0,0.210135
202211,167168.0,0.210021,334427.0,0.420157
202212,158105.0,0.198635,492532.0,0.618791
202301,187094.0,0.235055,679626.0,0.853847
202302,116332.0,0.146153,795958.0,1.000000


In [17]:
SBFE_combined = SBFE_combined[SBFE_combined["dateadded"].astype(str).str.slice(0, 6) != "202210"]
temp_sample = SBFE_combined.sample(500_000, replace = False, random_state = 0, ignore_index = True)
print(temp_sample.shape)

(500000, 8)


In [18]:
temp_sample["business_fein"] = ""
temp_sample["source"] = "SBFE_combined"

#### BIID Sample

In [9]:
path = "Analytics/Personal Folders/liuwei01/2023/ARMBS_ticket/2390_SOS/prod_liuwei01_37109_biid2_input_W20230221-133008.csv"
biid = pd.read_adls(path, reader = pd.read_csv,  encoding='iso-8859-1', dtype = str)
print(biid.shape)

(7411276, 93)


In [11]:
keep = ['transaction_id', 'datetime', 'incompanyname', 'incompanystreetaddress', 'incompanycity','incompanystate', 'incompanyzip5', 'incompanyphone', "incompanyfein"]
biid = biid[keep]
biid.columns = ['transactionid', 'dateadded', 'businessname', 'businessaddress', 'businesscity','businessstate', 'businesszip', 'businessphone', "business_fein"]
biid["blank_ct"] = biid[['businessname', 'businessaddress', 'businesscity', 'businessstate']].isnull().sum(axis = 1)
freq(biid["blank_ct"])

/tmp/ipykernel_31424/767051593.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biid["blank_ct"] = biid[['businessname', 'businessaddress', 'businesscity', 'businessstate']].isnull().sum(axis = 1)


,Count,Pct,Cuml Count,Cuml Pct
blank_ct,,,,
0,6978629.0,9.416231e-01,6978629.0,0.941623
1,13883.0,1.873227e-03,6992512.0,0.943496
2,7477.0,1.008868e-03,6999989.0,0.944505
3,5.0,6.746477e-07,6999994.0,0.944506
4,411282.0,5.549409e-02,7411276.0,1.000000


In [12]:
biid = biid[biid["blank_ct"] == 0]
biid = biid.sort_values(by = ['businessname', 'businessaddress', 'businesscity','businessstate', 'dateadded'])
biid = biid.drop_duplicates(subset = ['businessname', 'businessaddress', 'businesscity','businessstate'], keep = "last", ignore_index = True)
print(biid.shape)

(5616978, 10)


In [13]:
freq(biid["dateadded"].astype(str).str.slice(0, 6))

,Count,Pct,Cuml Count,Cuml Pct
dateadded,,,,
202210,1178829.0,0.209869,1178829.0,0.209869
202211,1143592.0,0.203596,2322421.0,0.413464
202212,1140410.0,0.203029,3462831.0,0.616494
202301,1308767.0,0.233002,4771598.0,0.849496
202302,845380.0,0.150504,5616978.0,1.000000


In [16]:
biid = biid[biid["dateadded"].astype(str).str.slice(0, 6) == "202302"]
temp_sample_biid = biid.sample(500_000, replace = False, random_state = 0, ignore_index = True)
print(temp_sample_biid.shape)
temp_sample_biid["source"] = "IIDBv2"

(500000, 10)


#### Final Sample

In [19]:
final = pd.concat([temp_sample, temp_sample_biid], ignore_index= True)
print(final.shape)
freq(final.source)

(1000000, 11)


,Count,Pct,Cuml Count,Cuml Pct
source,,,,
IIDBv2,500000.0,0.5,500000.0,0.5
SBFE_combined,500000.0,0.5,1000000.0,1.0


In [22]:
final.to_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2390/sbfe_combined_biid_1mil_2390_input.parquet", format = '.parquet', overwrite = True)

## Bullet 2 for engineering - get BIPIDs

In [2]:
## error in this round of corpkey pull
# path = 'Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2390/0912_ln_11605_20230318_busv31_credit_2390_corp_key_w20230911-171210.csv'
path = 'Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2390/0914_ln_11605_20230318_busv31_credit_2390_corp_key_w20230913-141308.csv'
corpkey = pd.read_adls(path, reader = pd.read_csv, encoding='iso-8859-1', dtype = str, delimiter = "|")
print(corpkey.shape)

(3162129, 266)


In [3]:
corpkey = corpkey[corpkey['seleid'].notnull()]
print(corpkey.shape)

(3162036, 266)


In [2]:
# path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2390/Old/ln_11605_20230318_busv31_credit_w20230320-120136_sas_layout_busshell.csv.gz"
# shell = pd.read_adls(path, reader = pd.read_csv, compression = "gzip", encoding='iso-8859-1', dtype = str, usecols = ["account", "id_seleid", "sos_inc_filing_count"])
# print(shell.shape)

(1000000, 3)


In [4]:
path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2390/0912_ln_11605_20230318_busv31_credit_w20230911-114434_sas_layout_busshell.csv.gz"
fields = ["account", "id_seleid", "history_datetime", "sos_inc_filing_count", 
          'in_bus_name', 'in_bus_alternatename', 'in_bus_streetaddress1', 'in_bus_streetaddress2', 'in_bus_city', 'in_bus_state', 'in_bus_zip', 
          'out_bus_prim_range', 'out_bus_predir', 'out_bus_prim_name', 'out_bus_addr_suffix', 'out_bus_postdir', 'out_bus_unit_desig', 'out_bus_sec_range', 'out_bus_state', 
          'out_bus_zip5', 'out_bus_zip4', 'best_bus_name', 'best_bus_streetaddress1', 'best_bus_city', 'best_bus_state', 'best_bus_zip', 'best_bus_fein', 'best_bus_phone10', ]
shell_new = pd.read_adls(path, reader = pd.read_csv, compression = 'gzip', encoding='iso-8859-1', dtype = str, usecols = fields)
print(shell_new.shape)

(1000000, 28)


In [7]:
# shell_new.columns = ["n_" + x for x in shell_new.columns]
# merged = shell.merge(shell_new, left_on = "account", right_on = "n_account")
# print(merged.shape)
# merged["flag"] = np.where(merged.id_seleid != merged.n_id_seleid, 1, 0)
# fmt = make_format(cuts = [-np.inf, 0, -1, np.inf])
# freq(merged["flag"], merged.id_seleid.astype(float), merged.n_id_seleid.astype(float), format = [None, fmt, fmt], observed = True, cross = False)

In [8]:
# merged["n_sos_inc_filing_count"] = merged["n_sos_inc_filing_count"].astype(float)
# fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, np.inf], exceptions = [-1])
# freq(merged["n_sos_inc_filing_count"], format = [fmt])

In [5]:
shell_new["sos_inc_filing_count"] = shell_new["sos_inc_filing_count"].astype(float)
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, np.inf], exceptions = [-1])
freq(shell_new["sos_inc_filing_count"], format = [fmt])

,Count,Pct,Cuml Count,Cuml Pct
sos_inc_filing_count,,,,
<= 0,205916.0,0.205916,205916.0,0.205916
1,410245.0,0.410245,616161.0,0.616161
2,58211.0,0.058211,674372.0,0.674372
3,13880.0,0.013880,688252.0,0.688252
4,5309.0,0.005309,693561.0,0.693561
5,2588.0,0.002588,696149.0,0.696149
6+,8073.0,0.008073,704222.0,0.704222
-1,295775.0,0.295775,999997.0,0.999997
Missing,3.0,0.000003,1000000.0,1.000000


In [6]:
shell_new[shell_new["sos_inc_filing_count"] >0].shape

(498306, 28)

In [7]:
corpkey_temp = corpkey[['seleid']].copy()
corpkey_temp = corpkey_temp.drop_duplicates(subset= ["seleid"])
print(corpkey_temp.shape)

shell_temp = shell_new[["account", "id_seleid", "sos_inc_filing_count"]].copy()
merged = shell_temp.merge(corpkey_temp, left_on = "id_seleid", right_on = "seleid", how = "left")
print(merged.shape)

(477030, 1)
(1000000, 4)


In [8]:
corpkey_temp.seleid.is_unique

True

In [9]:
merged[merged["seleid"].isnull() & (merged.sos_inc_filing_count >0)].shape

(647, 4)

In [10]:
647/498306

0.0012983989757297725

In [11]:
merged[merged["seleid"].isnull() & (merged.sos_inc_filing_count >0)].head()

,account,id_seleid,sos_inc_filing_count,seleid
262,172682321R5807,137961719219,1.0,NaN
1195,172864281R1141,123005557,1.0,NaN
3065,172508785R74544,139761799887,1.0,NaN
3641,172943411R603468,139231812068,1.0,NaN
4046,172911751R599290,43452060,1.0,NaN


## Analysis

#### BR Key

In [28]:
path = 'Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2390/0922_ln_11605_20230318_busv31_credit_2390_bus_reg_w20230919-150830.csv.gz'
BR_key = pd.read_adls(path, reader = pd.read_csv, encoding='iso-8859-1', dtype = str, compression = "gzip")
print(BR_key.shape)

(770950, 501)


In [29]:
BR_key[BR_key.seleid == "57529698"].tail()

,ultid,orgid,seleid,proxid,powid,empid,dotid,ultscore,orgscore,selescore,...,append_off3rawaid,append_off3aceaid,append_off4rawaid,append_off4aceaid,append_off5rawaid,append_off5aceaid,append_off6rawaid,append_off6aceaid,source_rec_id,fp
112127,57529698,57529698,57529698,0,0,0,0,98,98,98,...,0,0,0,0,0,0,0,0,2453370,0
112128,57529698,57529698,57529698,57529698,57529698,0,0,100,100,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
BR_key[BR_key.seleid.isnull()].shape

(57, 501)

In [31]:
BR_key = BR_key[BR_key["seleid"].notnull()]
print(BR_key.shape)

(770893, 501)


In [16]:
wb = TableWriter(filename = "./_temp/brkey.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)

wb.write_table(
    pd.DataFrame(BR_key.isnull().sum()),
    sheetname="fair_lending",
)
wb.close()

In [17]:
wb = TableWriter(filename = "./_temp/corpkey.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)

wb.write_table(
    pd.DataFrame(corpkey.isnull().sum()),
    sheetname="fair_lending",
)
wb.close()

#### Merge

In [32]:
corpkey_temp = corpkey[['seleid']].copy()
print(corpkey_temp.shape)
corpkey_temp = corpkey_temp.drop_duplicates(subset= ["seleid"])
print(corpkey_temp.shape)

brkey_temp = BR_key[['seleid']].copy()
print(brkey_temp.shape)
brkey_temp = brkey_temp.drop_duplicates(subset= ["seleid"])
print(brkey_temp.shape)

(3162036, 1)
(477030, 1)
(770893, 1)
(425991, 1)


In [33]:
brkey_temp.columns = ["br_seleid"]
merged = corpkey_temp.merge(brkey_temp, left_on = 'seleid', right_on = "br_seleid", how = "outer")
print(merged.shape)

(533293, 2)


In [34]:
merged["found"] = np.where(merged.seleid.notnull(), np.where(merged.br_seleid.notnull(), "both", "Corp only"), 
                           np.where(merged.br_seleid.notnull(), "BR only", "None"))
freq(merged["found"])

,Count,Pct,Cuml Count,Cuml Pct
found,,,,
BR only,56263.0,0.105501,56263.0,0.105501
Corp only,107302.0,0.201206,163565.0,0.306708
both,369728.0,0.693292,533293.0,1.000000


### Focus on BR only

In [35]:
br_only = merged[merged["found"] == "BR only"]
print(br_only.shape)

(56263, 3)


In [36]:
br_only_list = br_only["br_seleid"].to_list()

In [66]:
shell_new_filter = shell_new[shell_new["id_seleid"].isin(br_only_list)]
print(shell_new_filter.shape)

(62156, 28)


In [69]:
shell_list = shell_new_filter["id_seleid"].to_list()
[x for x in br_only_list if x not in shell_list]

['ROCHESTER', 'OSSIPEE', '1018A ROUTE 16 WHITE MOUNTAIN HIGHWAY']

In [68]:
shell_new_filter = shell_new_filter.sort_values(by = ["id_seleid"]).reset_index(drop = True)
# shell_new_filter[shell_new_filter.duplicated('id_seleid', keep=False) == True]
shell_new_filter = shell_new_filter.drop_duplicates(subset = ["id_seleid", "best_bus_name", "best_bus_streetaddress1", "best_bus_city", 'best_bus_state', "best_bus_zip"])
print(shell_new_filter.shape)

(56260, 28)


In [47]:
BR_key_filter = BR_key[BR_key["seleid"].isin(br_only_list)]
print(BR_key_filter.shape)

(80298, 501)


In [44]:
BR_key[BR_key["seleid"] == "62295"]

,ultid,orgid,seleid,proxid,powid,empid,dotid,ultscore,orgscore,selescore,proxscore,powscore,empscore,dotscore,ultweight,orgweight,seleweight,proxweight,powweight,empweight,dotweight,bdid,bdid_score,br_id,dt_first_seen,dt_last_seen,dt_vendor_first_reported,dt_vendor_last_reported,record_type,record_date,rawfields.ofc1_name,rawfields.ofc1_title,rawfields.first,rawfields.mi,rawfields.last,rawfields.suffix,rawfields.gender,rawfields.ownr_code,rawfields.xcode,rawfields.company,rawfields.mail_add,rawfields.mail_suite,rawfields.mail_city,rawfields.mail_state,rawfields.mail_zip,rawfields.mail_zip4,rawfields.mail_key,rawfields.county,rawfields.country,rawfields.district,rawfields.citylimits,rawfields.biz_ac,rawfields.biz_phone,rawfields.fax_ac,rawfields.fax_num,rawfields.sic,rawfields.naics,rawfields.descript,rawfields.emp_size,rawfields.own_size,rawfields.corpcode,rawfields.sos_code,rawfields.filing_cod,rawfields.state_code,rawfields.status,rawfields.stat_des,rawfields.lic_sts,rawfields.lic_type,rawfields.filing_num,rawfields.licid,rawfields.acct_num,rawfields.co_fei,rawfields.ctrl_num,rawfields.start_date,rawfields.file_date,rawfields.ccyymmdd,rawfields.form_date,rawfields.exp_date,rawfields.disol_date,rawfields.rpt_date,rawfields.renew_date,rawfields.chang_date,rawfields.appt_date,rawfields.fisc_date_,rawfields.duration,rawfields.loc_add,rawfields.loc_suite,rawfields.loc_city,rawfields.loc_state,rawfields.loc_zip,rawfields.loc_zip4,rawfields.loc_idx,rawfields.ofc1_add,rawfields.ofc1_suite,rawfields.ofc1_city,rawfields.ofc1_state,rawfields.ofc1_zip,rawfields.ofc1_ac,rawfields.ofc1_phone,rawfields.ofc1_fein,rawfields.ofc1_ssn,rawfields.ofc1_type,rawfields.ofc2_name,rawfields.ofc2_title,rawfields.ofc2_add,rawfields.ofc2_csz,rawfields.ofc2_fein,rawfields.ofc2_ssn,rawfields.ofc3_name,rawfields.ofc3_title,rawfields.ofc3_add,rawfields.ofc3_csz,rawfields.ofc3_fein,rawfields.ofc3_ssn,rawfields.ofc4_name,rawfields.ofc4_title,rawfields.ofc4_add,rawfields.ofc4_csz,rawfields.ofc4_fein,rawfields.ofc4_ssn,rawfields.ofc5_name,rawfields.ofc5_title,rawfields.ofc5_add,rawfields.ofc5_csz,rawfields.ofc5_fein,rawfields.ofc5_ssn,rawfields.ofc6_name,rawfields.ofc6_title,rawfields.ofc6_add,rawfields.ofc6_csz,rawfields.ofc6_fein,rawfields.ofc6_ssn,rawfields.ra_date,rawfields.ra_file,rawfields.ra_name,rawfields.ra_add,rawfields.ra_suite,rawfields.ra_city,rawfields.ra_state,rawfields.ra_zip,rawfields.ra_ac,rawfields.ra_phone,rawfields.class,rawfields.stock_iss,rawfields.stock_par,rawfields.stock_type,rawfields.stock_cap,rawfields.paidn_cap,rawfields.fee,rawfields.fee_2,rawfields.fee_3,rawfields.tax_cl,rawfields.act,rawfields.chapter,rawfields.page,rawfields.volume,rawfields.comments,rawfields.email,rawfields.user_name,rawfields.dsf,rawfields.hmbase,rawfields.ho,rawfields.solicit,rawfields.fips,rawfields.record_no,rawfields.misc_code,rawfields.agent_key,rawfields.proc_date,rawfields.crlf,clean_officer1_name.title,clean_officer1_name.fname,clean_officer1_name.mname,clean_officer1_name.lname,clean_officer1_name.name_suffix,clean_officer1_name.name_score,clean_officer2_name.title,clean_officer2_name.fname,clean_officer2_name.mname,clean_officer2_name.lname,clean_officer2_name.name_suffix,clean_officer2_name.name_score,clean_officer3_name.title,clean_officer3_name.fname,clean_officer3_name.mname,clean_officer3_name.lname,clean_officer3_name.name_suffix,clean_officer3_name.name_score,clean_officer4_name.title,clean_officer4_name.fname,clean_officer4_name.mname,clean_officer4_name.lname,clean_officer4_name.name_suffix,clean_officer4_name.name_score,clean_officer5_name.title,clean_officer5_name.fname,clean_officer5_name.mname,clean_officer5_name.lname,clean_officer5_name.name_suffix,clean_officer5_name.name_score,clean_officer6_name.title,clean_officer6_name.fname,clean_officer6_name.mname,clean_officer6_name.lname,clean_officer6_name.name_suffix,clean_officer6_name.name_score,clean_mailing_address.prim_range,clean_mailing_address.predir,clean_mailing_address.prim_nam

In [48]:
pd.set_option('display.max_columns', None)
BR_key_filter = BR_key_filter[["seleid", 'dt_first_seen', 'dt_last_seen', 'rawfields.company', 'rawfields.mail_add', 'rawfields.mail_suite', 'rawfields.mail_city', 'rawfields.mail_state', 'rawfields.mail_zip', 
               'clean_mailing_address.prim_range', 'clean_mailing_address.predir', 'clean_mailing_address.prim_name', 'clean_mailing_address.addr_suffix', 
               'clean_mailing_address.postdir', 'clean_mailing_address.unit_desig', 'clean_mailing_address.sec_range', 'clean_mailing_address.p_city_name', 
               'clean_mailing_address.v_city_name', 'clean_mailing_address.st', 'clean_mailing_address.zip', "clean_mailing_address.zip4"]]
BR_key_filter.head()

,seleid,dt_first_seen,dt_last_seen,rawfields.company,rawfields.mail_add,rawfields.mail_suite,rawfields.mail_city,rawfields.mail_state,rawfields.mail_zip,clean_mailing_address.prim_range,clean_mailing_address.predir,clean_mailing_address.prim_name,clean_mailing_address.addr_suffix,clean_mailing_address.postdir,clean_mailing_address.unit_desig,clean_mailing_address.sec_range,clean_mailing_address.p_city_name,clean_mailing_address.v_city_name,clean_mailing_address.st,clean_mailing_address.zip,clean_mailing_address.zip4
157,28021,20160330,20160330,A & B IRON & METAL CO INC,329 WASHINGTON ST,NaN,DAYTON,OH,45402,329,NaN,WASHINGTON,ST,NaN,NaN,NaN,DAYTON,DAYTON,OH,45402,2541
202,41257,20220725,20220725,NJ STATE BAR ASSN,1 CONSTITUTION SQ,NaN,NEW BRUNSWICK,NJ,8901,1,NaN,CONSTITUTION,SQ,NaN,NaN,NaN,NEW BRUNSWICK,NEW BRUNSWICK,NJ,8901,1587
222,51093,20090628,20090628,020 SQUAD CLUB,1801 9TH ST,NaN,SACRAMENTO,CA,95811,1801,NaN,9TH,ST,NaN,NaN,NaN,SACRAMENTO,SACRAMENTO,CA,95811,7011
236,62295,20070223,20070223,1-800-GIFT CERTIFICATE LLC,160 CHUBB AVE,SUITE 203,LYNDHURST,NJ,7071,160,NaN,CHUBB,AVE,NaN,STE,203,LYNDHURST,LYNDHURST,NJ,7071,3526
237,62295,20070223,20070223,1-800-GIFT CERTIFICATE LLC,160 CHUBB AVE,SUITE 203,LYNDHURST,NJ,7071,160,NaN,CHUBB,AVE,NaN,STE,203,LYNDHURST,LYNDHURST,NJ,7071,3526


In [53]:
BR_key_filter = BR_key_filter[~BR_key_filter.seleid.isin(['ROCHESTER', 'OSSIPEE', '1018A ROUTE 16 WHITE MOUNTAIN HIGHWAY'])]
print(BR_key_filter.shape)
BR_key_filter = BR_key_filter.sort_values(by = ['seleid', 'dt_last_seen'], ascending = False, ignore_index = True).reset_index(drop = True)

(80241, 21)


In [54]:
BR_key_filter.head()

,seleid,dt_first_seen,dt_last_seen,rawfields.company,rawfields.mail_add,rawfields.mail_suite,rawfields.mail_city,rawfields.mail_state,rawfields.mail_zip,clean_mailing_address.prim_range,clean_mailing_address.predir,clean_mailing_address.prim_name,clean_mailing_address.addr_suffix,clean_mailing_address.postdir,clean_mailing_address.unit_desig,clean_mailing_address.sec_range,clean_mailing_address.p_city_name,clean_mailing_address.v_city_name,clean_mailing_address.st,clean_mailing_address.zip,clean_mailing_address.zip4
0,99999266750,20160421,20160421,RAD MEDIA CO,11801 FAIR OAKS BLVD APT 13,NaN,FAIR OAKS,CA,95628,11801,NaN,FAIR OAKS,BLVD,NaN,APT,13,FAIR OAKS,FAIR OAKS,CA,95628,2846
1,99997142350,20160405,20160405,RED APPLE DELI CORP,294 E 204TH ST,NaN,BRONX,NY,10467,294,E,204TH,ST,NaN,NaN,NaN,BRONX,BRONX,NY,10467,4102
2,99995141150,20160406,20160406,MYJ NAILS INC,240 HARRISON AVE,NaN,HARRISON,NY,10528,240,NaN,HARRISON,AVE,NaN,NaN,NaN,HARRISON,HARRISON,NY,10528,4302
3,99995088350,20190308,20190308,ORANGE PROPERTY RESOURCES LLC,101 W MANLIUS ST,NaN,EAST SYRACUSE,NY,13057,101,W,MANLIUS,ST,NaN,NaN,NaN,EAST SYRACUSE,EAST SYRACUSE,NY,13057,2554
4,99995088350,20160329,20160329,ORANGE PROPERTY RESOURCES LLC,2054 MAIN ST,NaN,NEW WOODSTOCK,NY,13122,2054,NaN,MAIN,ST,NaN,NaN,NaN,NEW WOODSTOCK,NEW WOODSTOCK,NY,13122,9757


In [71]:
shell_new_filter = shell_new_filter.merge(BR_key_filter, left_on = "id_seleid", right_on = "seleid")
print(shell_new_filter.shape)

(80241, 49)


In [117]:
from difflib import SequenceMatcher

In [144]:
shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["clean_mailing_address.prim_range"].isnull(), "" , shell_new_filter["clean_mailing_address.prim_range"].str.lower().str.replace('s/+',"")) +\
    np.where(shell_new_filter["clean_mailing_address.predir"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.predir"].str.lower().str.replace('s/+',"") + " ") + \
    np.where(shell_new_filter["clean_mailing_address.prim_name"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.prim_name"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.addr_suffix"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.addr_suffix"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.postdir"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.postdir"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.unit_desig"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.unit_desig"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.sec_range"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.sec_range"].str.lower().str.replace('s/+',"")) 
    

/tmp/ipykernel_13864/3155675607.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["clean_mailing_address.prim_range"].isnull(), "" , shell_new_filter["clean_mailing_address.prim_range"].str.lower().str.replace('s/+',"")) +\
/tmp/ipykernel_13864/3155675607.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter["clean_mailing_address.predir"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.predir"].str.lower().str.replace('s/+',"") + " ") + \
/tmp/ipykernel_13864/3155675607.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter["clean_mailing_address.prim_name"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.prim_name"].str.lower().str.replace('s/+',"")) + \
/tmp/ipykernel_13864/3155675607.py:4: FutureW

In [145]:
# BR key cleaned mailing address vs best address
shell_new_filter["best_bus_streetaddress1"] = shell_new_filter.best_bus_streetaddress1.str.lower().str.replace('s/+',"")
shell_new_filter = shell_new_filter.applymap(lambda x: x.strip() if isinstance(x, str) else x)
shell_new_filter = shell_new_filter.applymap(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

shell_new_filter["name_same"] = np.where(shell_new_filter["rawfields.company"].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_name.str.lower().str.replace('s/+',""), 1 , 0)
shell_new_filter["addr_same"] = np.where(shell_new_filter["key_cleaned_addr"] == shell_new_filter.best_bus_streetaddress1, 1, 0)
shell_new_filter["city_same"] = np.where(shell_new_filter["clean_mailing_address.p_city_name"].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_city.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["state_same"] = np.where(shell_new_filter["clean_mailing_address.st"].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_state.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["zip_same"] = np.where(shell_new_filter["clean_mailing_address.zip"].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_zip.str.lower().str.replace('s/+',""), 1, 0)

shell_new_filter["best_bus_streetaddress1"] = shell_new_filter.best_bus_streetaddress1.astype(str)
shell_new_filter["key_cleaned_addr"] = shell_new_filter.key_cleaned_addr.astype(str)

for indx, row in shell_new_filter.iterrows():
    ratio = SequenceMatcher(None, row["best_bus_streetaddress1"], row["key_cleaned_addr"]).ratio() 
    shell_new_filter.loc[indx, "similarity"] = ratio
    
shell_new_filter["br_clean_vs_best"] = np.where((((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1) & (shell_new_filter["zip_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["zip_same"] ==1)), 1, 0)

freq(shell_new_filter["br_clean_vs_best"])

/tmp/ipykernel_13864/689086333.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["best_bus_streetaddress1"] = shell_new_filter.best_bus_streetaddress1.str.lower().str.replace('s/+',"")
/tmp/ipykernel_13864/689086333.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["name_same"] = np.where(shell_new_filter["rawfields.company"].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_name.str.lower().str.replace('s/+',""), 1 , 0)
/tmp/ipykernel_13864/689086333.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["city_same"] = np.where(shell_new_filter["clean_mailing_address.p_city_name"].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_city.str.lower().str.replace('s/+',""), 1, 0)
/tmp/ipykernel_13864/689086333.py:9: FutureWarning: The default value of regex will cha

,Count,Pct,Cuml Count,Cuml Pct
br_clean_vs_best,,,,
0,6384.0,0.07956,6384.0,0.07956
1,73857.0,0.92044,80241.0,1.00000


In [146]:
# BR key raw mailing address vs best address
shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["rawfields.mail_add"].isnull(), "", shell_new_filter["rawfields.mail_add"].str.lower().str.replace('s/+',"")) +\
    np.where(shell_new_filter["rawfields.mail_suite"].isnull(), "", " " + shell_new_filter["rawfields.mail_suite"].str.lower().str.replace('s/+',""))
    
shell_new_filter["addr_same"] = np.where(shell_new_filter["key_cleaned_addr"] == shell_new_filter.best_bus_streetaddress1, 1, 0)
shell_new_filter["city_same"] = np.where(shell_new_filter['rawfields.mail_city'].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_city.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["state_same"] = np.where(shell_new_filter['rawfields.mail_state'].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_state.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["zip_same"] = np.where(shell_new_filter['rawfields.mail_zip'].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_zip.str.lower().str.replace('s/+',""), 1, 0)

shell_new_filter["best_bus_streetaddress1"] = shell_new_filter.best_bus_streetaddress1.astype(str)
shell_new_filter["key_cleaned_addr"] = shell_new_filter.key_cleaned_addr.astype(str)

for indx, row in shell_new_filter.iterrows():
    ratio = SequenceMatcher(None, row["best_bus_streetaddress1"], row["key_cleaned_addr"]).ratio() 
    shell_new_filter.loc[indx, "similarity"] = ratio
    
shell_new_filter["br_raw_vs_best"] = np.where((((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1) & (shell_new_filter["zip_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["zip_same"] ==1)), 1, 0)

freq(shell_new_filter["br_raw_vs_best"])

/tmp/ipykernel_13864/1600107176.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["rawfields.mail_add"].isnull(), "", shell_new_filter["rawfields.mail_add"].str.lower().str.replace('s/+',"")) +\
/tmp/ipykernel_13864/1600107176.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter["rawfields.mail_suite"].isnull(), "", " " + shell_new_filter["rawfields.mail_suite"].str.lower().str.replace('s/+',""))
/tmp/ipykernel_13864/1600107176.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["city_same"] = np.where(shell_new_filter['rawfields.mail_city'].str.lower().str.replace('s/+',"") == shell_new_filter.best_bus_city.str.lower().str.replace('s/+',""), 1, 0)
/tmp/ipykernel_13864/1600107176.py:7: FutureWarning: The default value of regex will

,Count,Pct,Cuml Count,Cuml Pct
br_raw_vs_best,,,,
0,6880.0,0.085742,6880.0,0.085742
1,73361.0,0.914258,80241.0,1.000000


In [148]:
shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["clean_mailing_address.prim_range"].isnull(), "" , shell_new_filter["clean_mailing_address.prim_range"].str.lower().str.replace('s/+',"")) +\
    np.where(shell_new_filter["clean_mailing_address.predir"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.predir"].str.lower().str.replace('s/+',"") + " ") + \
    np.where(shell_new_filter["clean_mailing_address.prim_name"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.prim_name"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.addr_suffix"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.addr_suffix"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.postdir"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.postdir"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.unit_desig"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.unit_desig"].str.lower().str.replace('s/+',"")) + \
    np.where(shell_new_filter["clean_mailing_address.sec_range"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.sec_range"].str.lower().str.replace('s/+',"")) 

/tmp/ipykernel_13864/3866371369.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["clean_mailing_address.prim_range"].isnull(), "" , shell_new_filter["clean_mailing_address.prim_range"].str.lower().str.replace('s/+',"")) +\
/tmp/ipykernel_13864/3866371369.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter["clean_mailing_address.predir"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.predir"].str.lower().str.replace('s/+',"") + " ") + \
/tmp/ipykernel_13864/3866371369.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter["clean_mailing_address.prim_name"].isnull(), "" , " " + shell_new_filter["clean_mailing_address.prim_name"].str.lower().str.replace('s/+',"")) + \
/tmp/ipykernel_13864/3866371369.py:4: FutureW

In [149]:
# BR key cleaned mailing address vs input address
shell_new_filter["input_addr"] = np.where(shell_new_filter.in_bus_streetaddress1.isnull(), "", shell_new_filter.in_bus_streetaddress1.astype(str).str.lower().str.replace('s/+',"")) +\
    np.where(shell_new_filter.in_bus_streetaddress2.isnull(), "", " " + shell_new_filter.in_bus_streetaddress2.astype(str).str.lower().str.replace('s/+',""))
    
shell_new_filter = shell_new_filter.applymap(lambda x: x.strip() if isinstance(x, str) else x)
shell_new_filter = shell_new_filter.applymap(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

shell_new_filter["addr_same"] = np.where(shell_new_filter["key_cleaned_addr"] == shell_new_filter.input_addr, 1, 0)
shell_new_filter["city_same"] = np.where(shell_new_filter["clean_mailing_address.p_city_name"].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_city.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["state_same"] = np.where(shell_new_filter["clean_mailing_address.st"].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_state.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["zip_same"] = np.where(shell_new_filter["clean_mailing_address.zip"].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_zip.str.lower().str.replace('s/+',""), 1, 0)

shell_new_filter["input_addr"] = shell_new_filter.input_addr.astype(str)
shell_new_filter["key_cleaned_addr"] = shell_new_filter.key_cleaned_addr.astype(str)

for indx, row in shell_new_filter.iterrows():
    ratio = SequenceMatcher(None, row["input_addr"], row["key_cleaned_addr"]).ratio() 
    shell_new_filter.loc[indx, "similarity"] = ratio

shell_new_filter["br_clean_vs_input"] = np.where((((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1) & (shell_new_filter["zip_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["zip_same"] ==1)), 1, 0)

freq(shell_new_filter["br_clean_vs_input"])

/tmp/ipykernel_13864/431398806.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["input_addr"] = np.where(shell_new_filter.in_bus_streetaddress1.isnull(), "", shell_new_filter.in_bus_streetaddress1.astype(str).str.lower().str.replace('s/+',"")) +\
/tmp/ipykernel_13864/431398806.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter.in_bus_streetaddress2.isnull(), "", " " + shell_new_filter.in_bus_streetaddress2.astype(str).str.lower().str.replace('s/+',""))
/tmp/ipykernel_13864/431398806.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["city_same"] = np.where(shell_new_filter["clean_mailing_address.p_city_name"].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_city.str.lower().str.replace('s/+',""), 1, 0)
/tmp/ipykernel_13864/431398806.py:10: FutureWarning: The defa

,Count,Pct,Cuml Count,Cuml Pct
br_clean_vs_input,,,,
0,14110.0,0.175845,14110.0,0.175845
1,66131.0,0.824155,80241.0,1.000000


In [150]:
# BR key raw mailing address vs best address
shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["rawfields.mail_add"].isnull(), "", shell_new_filter["rawfields.mail_add"].str.lower().str.replace('s/+',"")) +\
    np.where(shell_new_filter["rawfields.mail_suite"].isnull(), "", " " + shell_new_filter["rawfields.mail_suite"].str.lower().str.replace('s/+',""))
    
shell_new_filter["addr_same"] = np.where(shell_new_filter["key_cleaned_addr"] == shell_new_filter.input_addr, 1, 0)
shell_new_filter["city_same"] = np.where(shell_new_filter['rawfields.mail_city'].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_city.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["state_same"] = np.where(shell_new_filter['rawfields.mail_state'].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_state.str.lower().str.replace('s/+',""), 1, 0)
shell_new_filter["zip_same"] = np.where(shell_new_filter['rawfields.mail_zip'].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_zip.str.lower().str.replace('s/+',""), 1, 0)

shell_new_filter["best_bus_streetaddress1"] = shell_new_filter.best_bus_streetaddress1.astype(str)
shell_new_filter["key_cleaned_addr"] = shell_new_filter.key_cleaned_addr.astype(str)

for indx, row in shell_new_filter.iterrows():
    ratio = SequenceMatcher(None, row["input_addr"], row["key_cleaned_addr"]).ratio() 
    shell_new_filter.loc[indx, "similarity"] = ratio
    
shell_new_filter["br_raw_vs_input"] = np.where((((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1) & (shell_new_filter["zip_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1)) | (((shell_new_filter["similarity"] > 0.6) | (shell_new_filter["addr_same"] ==1)) & (shell_new_filter["zip_same"] ==1)), 1, 0)

freq(shell_new_filter["br_raw_vs_input"])

/tmp/ipykernel_13864/3782712212.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["key_cleaned_addr"] = np.where(shell_new_filter["rawfields.mail_add"].isnull(), "", shell_new_filter["rawfields.mail_add"].str.lower().str.replace('s/+',"")) +\
/tmp/ipykernel_13864/3782712212.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  np.where(shell_new_filter["rawfields.mail_suite"].isnull(), "", " " + shell_new_filter["rawfields.mail_suite"].str.lower().str.replace('s/+',""))
/tmp/ipykernel_13864/3782712212.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  shell_new_filter["city_same"] = np.where(shell_new_filter['rawfields.mail_city'].str.lower().str.replace('s/+',"") == shell_new_filter.in_bus_city.str.lower().str.replace('s/+',""), 1, 0)
/tmp/ipykernel_13864/3782712212.py:7: FutureWarning: The default value of regex will c

,Count,Pct,Cuml Count,Cuml Pct
br_raw_vs_input,,,,
0,14384.0,0.17926,14384.0,0.17926
1,65857.0,0.82074,80241.0,1.00000


In [143]:
shell_new_filter["addr_smilar"] = np.where(shell_new_filter.similarity > 0.6 , 1, 0)

freq(shell_new_filter.addr_smilar, shell_new_filter["addr_same"], shell_new_filter["city_same"], 
     shell_new_filter["state_same"], shell_new_filter["zip_same"], cross = False, observed = True)

Count       Pct  \
addr_smilar addr_same city_same state_same zip_same                      
1           1         1         0          1          4200.0  0.052342   
                                           0           203.0  0.002530   
                                1          1         33102.0  0.412532   
                                           0          1241.0  0.015466   
                      0         0          1            95.0  0.001184   
                                           0           101.0  0.001259   
                                1          1          1720.0  0.021435   
                                           0           185.0  0.002306   
            0         1         0          1          2263.0  0.028203   
                                           0           118.0  0.001471   
                                1          1         20154.0  0.251168   
                                           0          1380.0  0.017198   
                      0         0          1           104.0  0.001296   
                                           0           110.0  0.001371   
                                1          1          1598.0  0.019915   
                                           0           314.0  0.003913   
0           0         1         0          1           431.0  0.005371   
                                           0           180.0  0.002243   
                                1          1          3794.0  0.047283   
                                           0          2299.0  0.028651   
                      0         0          1            16.0  0.000199   
                                           0          1719.0  0.021423   
                                1          1           236.0  0.002941   
                                           0          4678.0  0.058299   

                                                     Cuml Count  Cuml Pct  
addr_smilar addr_same city_same state_same zip_same                        
1           1         1         0          1             4200.0  0.052342  
                                           0             4403.0  0.054872  
                                1          1            37505.0  0.467404  
                                           0            38746.0  0.482870  
                      0         0          1            38841.0  0.484054  
                                           0            38942.0  0.485313  
                                1          1            40662.0  0.506748  
                                           0            40847.0  0.509054  
            0         1         0          1            43110.0  0.537257  
                                           0            43228.0  0.538727  
                                1          1            63382.0  0.789895  
                                           0            64762.0  0.807094  
                      0         0          1            64866.0  0.808390  
                                           0            64976.0  0.809761  
                                1          1            66574.0  0.829676  
                                           0            66888.0  0.833589  
0           0         1         0          1            67319.0  0.838960  
                                           0            67499.0  0.841203  
                                1          1            71293.0  0.888486  
                                           0            73592.0  0.917137  
                      0         0          1            73608.0  0.917337  
                                           0            75327.0  0.938759  
                                1          1            75563.0  0.941701  
                                           0            80241.0  1.000000

In [ ]:
shell_new_filter["br_clean_vs_best"], shell_new_filter["br_raw_vs_best"], shell_new_filter["br_clean_vs_input"], shell_new_filter["br_raw_vs_input"]

In [152]:
roll = shell_new_filter.groupby("seleid")["br_clean_vs_best", "br_raw_vs_best", "br_clean_vs_input", "br_raw_vs_input"].max()
print(roll.shape)

(56260, 4)


/tmp/ipykernel_13864/844886191.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  roll = shell_new_filter.groupby("seleid")["br_clean_vs_best", "br_raw_vs_best", "br_clean_vs_input", "br_raw_vs_input"].max()


In [154]:
freq("br_clean_vs_best", df = roll, cross = False, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
br_clean_vs_best,,,,
1,53857.0,0.957288,53857.0,0.957288
0,2403.0,0.042712,56260.0,1.000000


In [155]:
freq("br_raw_vs_best", df = roll, cross = False, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
br_raw_vs_best,,,,
1,53596.0,0.952648,53596.0,0.952648
0,2664.0,0.047352,56260.0,1.000000


In [156]:
freq("br_clean_vs_input", df = roll, cross = False, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
br_clean_vs_input,,,,
1,48816.0,0.867686,48816.0,0.867686
0,7444.0,0.132314,56260.0,1.000000


In [157]:
freq("br_raw_vs_input", df = roll, cross = False, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
br_raw_vs_input,,,,
1,48717.0,0.865926,48717.0,0.865926
0,7543.0,0.134074,56260.0,1.000000


In [135]:
### debug
# shell_new_filter[(shell_new_filter["name_same"] ==0) & (shell_new_filter["addr_same"] ==1) & (shell_new_filter["city_same"] ==1) & 
#      (shell_new_filter["state_same"] ==1) & (shell_new_filter["zip_same"] ==1)].head()
temp = shell_new_filter.loc[(shell_new_filter["addr_same"] ==0) & (shell_new_filter["city_same"] ==1) & 
     (shell_new_filter["state_same"] ==1) & (shell_new_filter["zip_same"] ==1), ["key_cleaned_addr", 'best_bus_streetaddress1', "similarity"]].copy()

In [136]:
temp.head()

,key_cleaned_addr,best_bus_streetaddress1,similarity
37,5849 sawmill rd & patio,5849 sawmill rd,0.789474
46,18 peachtree ave ne apt 6,2870 peachtree rd nw,0.622222
63,2737 n kimball ave # 2,2737 n kimball ave,0.900000
64,2737 n kimball ave # 2,2737 n kimball ave,0.900000
80,40 twin dr,3779 us route 9,0.240000


In [123]:
temp.iloc[1, 0]

'40 twin dr'

In [124]:
temp.iloc[1, 1]

'3779 us route 9'